## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-16-35-13 +0000,bbc,Trump says it is highly unlikely he will fire ...,https://www.bbc.com/news/articles/c4geyrdprwjo
1,2025-07-16-16-31-03 +0000,bbc,Labour suspends four rebel MPs for breaching p...,https://www.bbc.com/news/articles/c5y7zqdwzqyo
2,2025-07-16-16-18-35 +0000,bbc,Meet the sides bidding for a place in Wafcon 2...,https://www.bbc.com/sport/football/articles/c3...
3,2025-07-16-16-12-12 +0000,bbc,Badenoch and PM spar over 'modest income' tax ...,https://www.bbc.com/news/articles/cdx574e5yzlo
4,2025-07-16-16-12-00 +0000,wsj,Dollar Falls on Revived Threats to Powell,https://www.wsj.com/economy/central-banking/as...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2296/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,10
24,pm,5
72,trade,4
76,israel,4
53,china,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
44,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,33
33,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,29
45,2025-07-15-18-54-00 +0000,world,A Day After Trump Promised New Arms for Ukrain...,https://www.wsj.com/world/russia/trump-advised...,28
32,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,24
0,2025-07-16-16-35-13 +0000,bbc,Trump says it is highly unlikely he will fire ...,https://www.bbc.com/news/articles/c4geyrdprwjo,21


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
44,33,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
13,21,2025-07-16-14-28-04 +0000,bbc,Canada's PM wants to fast-track 'nation buildi...,https://www.bbc.com/news/articles/cew0kw29lvro
33,19,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
41,18,2025-07-15-20-10-21 +0000,bbc,Three key questions after Afghan data breach s...,https://www.bbc.com/news/articles/c9w12kdg5zko
25,17,2025-07-16-06-35-00 +0000,wsj,U.K. Inflation Heats Up as Bank of England Sta...,https://www.wsj.com/economy/u-k-inflation-heat...
11,14,2025-07-16-14-36-00 +0000,wsj,EU Exports to U.S. Cool as Trans-Atlantic Trad...,https://www.wsj.com/economy/trade/eu-exports-t...
36,13,2025-07-15-21-44-50 +0000,bbc,Israel bombs Syrian forces entering Druze city...,https://www.bbc.com/news/articles/c89e3j4e911o
16,12,2025-07-16-12-20-00 +0000,world,Nvidia CEO Lavishes Praise on China in Beijing...,https://www.wsj.com/world/asia/nvidia-ceo-lavi...
40,12,2025-07-15-20-17-00 +0000,wsj,Leading Economies Must Work Together to Fix Im...,https://www.wsj.com/economy/global/leading-eco...
45,10,2025-07-15-18-54-00 +0000,world,A Day After Trump Promised New Arms for Ukrain...,https://www.wsj.com/world/russia/trump-advised...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
